# Sprint 15 - Series Temporales (Sesiones)
**Versión para estudiantes**

Como científicos de datos, en muchas ocasiones se nos presenta el desafío de pronosticar una variable de la cual solamente conocemos su comportamiento a través del tiempo. A este tipo de variables se las conoce como **series temporales**, y en esta caso vamos a aprender cómo estudiarlas estadísticamente y generar a partir de aquí modelos predictivos para las mismas.

En concreto vamos a tratar los siguientes aspectos relevantes asociados a series temporales:

* Descomposición
* Estacionareidad
* Autocorrelación

Entendidos estos conceptos, los utilizaremos para extraer atributos que podamos incorporar como entradas para uno de los algoritmos que ya conocemos.

## Entendimiento del contexto

En la actualidad, uno de los aspectos más relevantes en el desarrollo de políticas públicas involucra garantizar un entorno amigable con el ambiente en las ciudades, que entre otras cosas conlleve mayor bienestar de sus habitantes actuales y futuros.  

Con esta visión en mente, el municipio de la ciudad de Quito te ha contratado para desarrollar un modelo predictivo que le permita pronosticar la calidad del aire en el centro de la urbe. En este sentido, se requiere que desarrolles una herramienta capaz de predecir la concentración de emisiones de monóxido de carbono (CO) para el día siguiente de las mediciones que se tomen con los sensores ya instalados. 

Estos resultados permitirán generar boletines preventivos para la ciudadanía, así como emitir nuevas ordenanzas para la gestión de contaminantes urbanos.

## Entendimiento de los datos

Carga las librerías y funciones con las que trabajarás en este caso, y entre las que debes incluir `LinearRegression`, `train_test_split` y `metrics`. Adicionalmente, de la librería **statsmodels** importa lo siguiente:

* `seasonal_decompose` del grupo `tsa.seasonal`.
* `pacf` y `adfuller` del grupo `tsa.stattools`.

Igualmente importa la librería **datetime** ya que nos ayudará a procesar las dimensiones temporales de nuestros datos. 

La entidad ha compartido contigo el dataset **emisiones_co**, que contiene más de 50,000 registros históricos levantados durante los últimos 6 años por los mencionados sensores de emisiones. Las columnas de esta tabla se describen a continuación:

* fecha: Fecha en la que se registró una medición de CO.
* hora: Hora en la que se registró una medición de CO.
* emisiones_co: Nivel de concentración de emisiones de CO en $mg/m^3$.

Carga y explora el dataset para que a partir de aquí definas un objetivo técnico, los algoritmos y métricas a utilizar, y un plan de acción para el procesamiento e ingeniería de datos.

**OBJETIVO TÉCNICO**

< Aquí tu repsuesta >

**ALGORITMO Y MÉTRICAS DE RENDIMIENTO**

< Aquí tu repsuesta >

**PLAN DE ACCION PARA PREPARACIÓN E INGENIERÍA DE DATOS**

< Aquí tu repsuesta >

## Preparación de datos

Limpia los datos conforme el plan de acción definido.

## Análisis de series temporales

Antes de iniciar con la construcción de un modelo predictivo para nuestro caso, conviene conocer las características distintivas de la serie temporal en cuestión. En concreto, nos interesa dar respuesta a las siguientes preguntas:

* ¿Cómo es la tendencia de las emisiones?
* ¿Existe alguna estacionalidad relevante en las emisiones?
* ¿Las emisiones pueden ser descritas como una función del tiempo?
* ¿Las emisiones pueden ser descritas como una función de su comportamiento pasado?

Dar solución a estas interrogantes es importante pues adicional a ayudarnos a comprender de mejor forma el comportamiento de neustra variable objetivo, nos permitirá crear los atributos necesarios para alimentar y entrenar el algoritmo de nuestro modelo.

Por tanto, en primer lugar conviene que visualices la serie temporal para comprender de mejor forma su evolución a través del tiempo.

Como puedes evidenciar en el gráfico anterior, describir el comportamiento de esta serie temporal puede ser complejo dados los continuos cambios a través del tiempo. Entonces, podría ser conveniente quizás enfocarnos en cada año para reducir la magnitud del problema. Visualiza la evolución de las emisiones en separado por todos los años existentes en la serie.

Un poco mejor, aunque aún es difícil encontrar patrones más generales del comportamiento de emisiones. Probemos ahora agregando temporalmente la serie para ver si se aprecia algo más. Visualiza la variable con una granularidad mensual (y ya no diaria). Aplica para esto el método `resample` de **pandas**. 

Hemos reducido la complejidad significativamente y ya podemos observar algunos comportamientos de mejor manera aunque los mismos no terminan siendo suficientemente claros. Intenta un mejor resultado con una agregación anual.

Ya se cuenta con una visión clara del comportamiento de nuesta serie aunque esto ha repercutido potencialmente en la pérdida de información relevante. Probemos entonces algo mejor que simplifique la comprension del comportamiento de la serie, pero que no implique sacrificar información. 

### Descomposición de series temporales

Las técnicas de visualización expuestas (particionamiento y agregación de la serie) facilitan la comprensión del comportamiento de una serie temporal, sin embargo no son suficientes para tener una perspectiva objetiva y clara de tendencias o estacionalidades. Es por esta razón que existe el método de **descomposión** y lo explicaremos a continuación.

Por definición, toda serie temporal puede descomponerse en los siguientes factores:

* El primero de ellos se denomina tendencia ($\tau$) y hace referencia al comportamiento de mediano/largo plazo de la serie.
* El siguiente es la estacionalidad ($\sigma$) que explica las fluctuaciones cíclicas en el corto plazo.
* Finalmente, queda el componente aleatorio o residuo ($\rho$), que comprende todas aquellas variaciones aleatorias o cuya explicación va más allá de los datos disponibles.

Es así que dada una serie temporal $y_t$, con $t \in T \subset \mathbb N$ como un índice que representa el tiempo, se cumple que

$$ y_i = \tau_i \times \sigma_i \times \rho_i\quad \forall i \in T $$

Regresemos a nuestra serie diaria y empieza obteniendo el componente de tendencia mediante el método `rolling` y guardando el resultado como una nueva columna. Conviene que trabajes con una copia de nuestra serie original para no alterar los datos originales y que visualices los resultados obtenidos. Visto que la agregación ha mostrado una mejor interpretabilidad a nivel anual, utiliza el argumento `window = 365`.

Lo que aquí has obtenido es también conocido como **media móvil** y en este caso no es más que un promedio de las 365 observaciones precedentes para cada registro de la serie de tiempo. Es decir, la media móvil $\bar y_i$ en el momento $i\in T$ se define por

$$ \bar y_i = \frac{1}{365}\sum_{k = 0}^{364} y_{i-k} $$

Notemos que la tendencia obtenida ya nos permite generar mejores conclusiones que cuando agregamos de forma temporal, simplificando de manera importante la complejidad de la serie pero sin que esto implique pérdida de información. En concreto, se puede observar que:

* El inicio de la pandemia del COVID generó una reducción de las emisiones en la ciudad, visto que hubo una cuarentena forzosa.
* Existe una tendencia de crecimiento constante hasta inicios de 2023 producto de la flexibilización de las medidas preventivas por la pandemia.
* Hay una reducción importante a partir del sengundo trimestre de 2023, y luego de esto un comportamiento estable. Se puede presumir que luego de la pandemia, se implementaron medidas de control por parte del municipio para controlar el incremento precedente.

Pasemos ahora a la estacionalidad $\sigma$. Obtenla siguiendo estos pasos:

1. En el dataset copiado que ya tienes, crea una columna donde dividas la serie original por la tendencia, y llámala "resto". 
2. Extrae el dia del año de la fecha y guárdalo en la columna "dia".
3. Agrupa los datos por dia y calcula la mediana de resto. Guarda este resultado en df_est.
4. Une df_est con el dataset copia utilizando como clave de union la columna dia.

Visualiza ahora esta estacionalidad con el dataset df_est.

Con este gráfico ya se puede afirmar lo siguiente respecto a comportamientos de corto plazo y que resulta consistente considerando las épocas de verano e invierno del año: 

* Existe una "temporada baja" de emisiones entre junio a octubre aproximadamente (mediados de cada año).
* Por su parte la "temporada alta" se ubica entre febrero a abril, y noviembre a diciembre (principios y finales de cada año). 

Obtén finalmente el componente residual $\rho$ dividiendo la columna resto por la estacionalidad, y visualízalo.

Como mencionamos, en el componente residual se explican todos aquellos factores externos y no observados que inciden en las emisiones en un momento del tiempo dado.

Toda esta descomposición puede hacerse automáticamente con la función `seasonal decompose`. Pruébala y evidencia que obtienes resultados bastante similares a los alcanzados en los puntos anteriores.

### Estacionareidad en series temporales

Una característica muy importante a nivel de series temporales hace referencia a la **estacionareidad**. Una serie es estacionaria si su valor promedio y su variabilidad son INDEPENDIENTES del tiempo. Cuando esto sucede, su grado de predictibilidad es mayor ya que es menos probable que existan atributos no observados que influyan en los valores de la misma.

De ser dependientes del tiempo, el promedio y la varianza de nuestra serie temporal $y_t$ pueden describirse como una función de este tiempo, tal que para todo $i \in T$

$$ E(y_i) = f(i) $$

$$ V(y_i) = E(y_i^2) - (E(y_i))^2 = h(i) $$

donde $f$ y $h$ son funciones definidas en los reales.

Intenta entonces verificar visualmente si existe alguna relación entre el promedio y el tiempo dada nuestra serie temporal de emisiones. Utiliza el concepto de media móvil al igual que antes.

Del gráfico anterior se desprende que no parece existir ninguna función conocida que describa este comportamiento a nivel del promedio de la serie. 

Has lo mismo pero ahora con la varianza.

Nuevamente no parece haber alguna relación funcional clara entre el tiempo y la variabilidad de las emisiones. Por tanto, existe evidencia a favor de la estacionareidad de la serie.

Estos resultados en todo caso no son concluyentes, puesto que no conocemos todas las funciones existentes y podríamos estar alcanzando una interpretación errónea. Justamente por esta razón es que se ha desarrollado la prueba **Dickey-Fuller** que evalúa las siguientes hipótesis:

* Hipótesis nula: La serie temporal NO es estacionaria
* Hipótesis alternativa: La serie temporal SÍ es estacionaria

Lleva a cabo esta prueba de hipótesis mediante la función `adfuller` y utiliza una significancia del 5%.

Queda confirmada estadísticamente nuestra suposición a partir de las visualizaciones, ¿pero qué sucede si no se evidencia estacionareidad en los datos? Tomemos un ejemplo muy básico que nos permita presentar el método de **integración** de series. Crea una serie de 100 observaciones que estén definidos en el tiempo de acuerdo a la función $f(i) = (i + \epsilon)^2$ de acuerdo al siguiente código:

```py
np.random.seed(123)
serie_ejemplo = pd.DataFrame(dict(
    datos = [(x + 10*np.random.randn())**2 for x in range(100)] 
))
```

Visualiza la evolución del promedio y la variabilidad de esta serie.

Ejecuta la prueba Dickey - Fuller para verificar la NO estacionareidad de esta serie.

Dado que esta serie NO es estacionaria, podemos transformarla mediante la operación llamada **integración** que se describe a continuación:

Sea $y_t$ una serie no estacionaria. Se cumple entonces que la serie $z_t = y_t - y_{t-1}$ tendrá mayor probabilidad de serlo.

Integra entonces la serie de ejemplo y verifica si la misma es estacionaria. Utiliza el método `shift` para desplazar la serie.

Fíjate que el valor p obtenido se ha reducido, lo cual es evidencia que la integración ayuda a generar estacionareidad. Este proceso puede repetirse las veces que sean necesarias hasta alcanzar la estacionareidad deseada. 

### Autocorrelación

La autocorrelación es una característica que hace referencia a la capacidad que tiene una observación pasada en la serie de pronosticar un valor futuro. En concreto, sea nuestra serie $y_t$, que tiene dos observaciones $y_i$ y $y_{i-k}$ donde $k > 0$ se denomina **lag**. Entonces, la autocorrelación corresponde a la magnitud en que $y_{i-k}$ puede explicar por sí misma el valor de $$y_i$.

En otras palabras, si existe autocorrelación entre una serie y uno de sus lags, el segundo podría utilizarse como atributo predictor en un modelo analítico.

Visto esto, estudia la autocorrelación de nuestra serie de emisiones mediante la función `pacf`.

De estos resultados se concluye que las emisiones de hasta una semana previa tiene capacidad de pronosticar el valor de la serie.

## Ingenieria de datos

Del análisis realizado se desprenden los siguientes atributos pontenciales a considerar en nuestro modelo predictivo:

* La tendencia y estacionalidad de la serie.
* Los lags de hasta 7 días previos, dado que existiría autocorrelación.

Créalos en el dataset de emisiones diarias.

Notemos dos cosas importantes:

* La tendencia y la estacionalidad presentan un potencial error puesto que no hace sentido que el valor de un atributo dependa del valor específico de la variable objetivo. Por tanto, para corregir esta inconsistencia debemos desplazar nuestros atributos con el método `shift`. De esta manera, cada observación ingresada al algoritmo del modelo tendrá como insumo información del día anterior y no del día que se busca pronosticar.
* Existen muchos valores perdidos en los atributos creados. No hace falta darles ningún tratamiento sino que preferentemente vamos a eliminarlos gracias a que tenemos suficientes registros en nuestro dataset.

Ejecuta todas estas acciones de manera ordenada.

Continua con la ingeniería separando el dataset en atributos y objetivo; reescalando la variable objetivo para que la misma siga una distribución aproximadamente normal; y posteriormente particionando los datos en conjuntos de entrenamiento y prueba. Para lo último considera que en series temporales es conveniente respetar el orden temporal de la información.

## Creación de modelo base

Crea y entrena un modelo base mediante el algoritmo de regresión lineal.

Evalúa el rendimiento del modelo base creado.

Estamos listos, hemos creado un modelo que es capaz de pronosticar las emisiones en la ciudad de Quito y ahora el municipio de la ciudad cuenta con una herramienta que le permite anticiparse al nivel de contaminacion por CO para tomar decisiones de política pública en beneficio de los habitantes de la urbe.

Para finalizar, en el siguiente código podrás ver cómo utilizar tu modelo para predecir las emisiones de CO de una semana en el futuro. Intenta mirar y comprender lo que hace.

```py
# Copiar el dataset original
df_pronostico = df_emisiones_d["emisiones"].copy()

# Generar pronósticos para la siguiente semana
for _ in range(7):

    # Extraer tendencia y estacionalidad
    descomp = seasonal_decompose(
        df_pronostico, 
        model = "multiplicative",
        period = 365,
        two_sided = False
    )

    # Crear nuevos atributos
    nuevos_atrib = pd.DataFrame(dict(
        tendencia = descomp.trend.iloc[-1],
        estacionalidad = descomp.seasonal.iloc[-1],
        lag_1 = df_pronostico.iloc[-1],
        lag_2 = df_pronostico.iloc[-2],
        lag_3 = df_pronostico.iloc[-3],
        lag_4 = df_pronostico.iloc[-4],
        lag_5 = df_pronostico.iloc[-5],
        lag_6 = df_pronostico.iloc[-6],
        lag_7 = df_pronostico.iloc[-7]
    ), index = [max(df_pronostico.index) + dt.timedelta(1)])

    # Generar prediccion con modelo creado
    y_pred = mod_reg.predict(nuevos_atrib)

    # Incorporar resultado en observaciones historicas
    df_aux = pd.Series(10**y_pred[0] - 1, index = [max(df_pronostico.index) + dt.timedelta(1)])

    df_pronostico = pd.concat(
        [df_pronostico, df_aux]
    )

# Visualizar pronostico realizado
plt.figure(figsize = [10,4])
plt.plot(df_pronostico[-14:-7], color = "dodgerblue", marker = ".", linestyle = "--")
plt.plot(df_pronostico[-8:], color = "tomato", marker = ".")
plt.ylabel("emisiones")
plt.grid(linestyle = ":")
plt.show()
```